# Import Packages

In [1]:
# built-in modules
import os

# third-party modules (pip install required)
import pandas as pd

# Load SRA Metadata

This is obtained from Step 1 of Modulome Workflow (https://github.com/avsastry/modulome-workflow/tree/main/1_download_metadata)

In [2]:
def data_dir(*file):
    return os.path.join('..', 'data', *file)

In [3]:
DF_metadata = pd.read_csv(data_dir('Escherichia_coli_20220127.tsv'), sep='\t', index_col=0)

In [4]:
DF_metadata.BioProject.value_counts().to_csv(data_dir('bioproject_list.csv'))

# Manually Curate BioProjects

- Annotate strain/substrain
- Remove non-RNA-seq or weird experimental design projects
- Attach publications, if any

# Use Curated BioProject List to Isolate K-12 BioProjects

In [5]:
DF_project_list = pd.read_csv(data_dir('bioproject_list_curated.csv'), index_col=0)
DF_project_list.head()

,Number of Samples,Reference Genome,Strain,Publication,PRECISE-1K,K-12 Modulome,Notes
BioProject,,,,,,,
PRJNA532938,318,U00096.2,W0153,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,False,False,NaN
PRJNA707564,165,NC_000913.3,RB001,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,False,False,Clinical isolates
PRJNA368968,152,NC_012967.1,B REL606,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,False,False,NaN
PRJNA604190,135,NC_000913.3,K-12 MG1655,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,False,True,NaN
PRJNA638918,108,Multiple,Multiple,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,False,False,Pathogens


In [14]:
# ensure we don't pick up datasets already in PRECISE-1K
k12_datasets = DF_project_list[(DF_project_list['K-12 Modulome']) & (~DF_project_list['PRECISE-1K'].fillna(False))].index
k12_metadata = DF_metadata[DF_metadata.BioProject.isin(k12_datasets)].copy()
# add columns we may use later
k12_metadata['R1'] = None
k12_metadata['R2'] = None
k12_metadata['swift'] = None
k12_metadata.to_csv(data_dir('K12', 'K12_metadata.tsv'), sep='\t')